## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embeddings, load one from gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2024-01-29 08:20:18--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2024-01-29 08:20:18--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfb7db0d5879cb9671216f4edb7.dl.dropboxusercontent.com/cd/0/get/CMQox_6aps6Ir5NkphEsE33G1d5n9Sp0b6hM7w3DsenuRO0Y37XsMv2JOOnw5G4kAoqZZGbeK1GN73izjC3qnRQ1ymCUaraWPb9AyryoJI1vMMQgGxeaJPveYF0lO6m_Jvo/file?dl=1# [following]
--2024-01-29 08:20:19--  https://ucfb7db0d5879cb9671216f4edb7.dl.dropboxusercontent.com/cd/0/get/CMQox_6aps6Ir5NkphEsE33G1d5n9Sp0b6hM7w3DsenuRO0Y37XsMv2JOOnw5G4kAoqZZGb

In [1]:
import numpy as np

with open("./quora.txt", encoding="utf-8") as file:
    data = list(file)

data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an NLP task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many NLP tasks like tokenization, stemming or part-of-speech tagging.

In [2]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize

from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

s = '''Good muffins cost $3.88\nin New York.  Please buy me two of them.\n\nThanks.'''
print(word_tokenize(s))

print(tokenizer.tokenize(s))


['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']
['Good', 'muffins', 'cost', '$', '3', '.', '88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']


In [4]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [5]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [word_tokenize(s.lower()) for s in data]

In [6]:
print(data_tok[50])

['what', 'tv', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'s", 'body', 'language', '?']


In [7]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [8]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy 's baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [9]:
%%time
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5)  # define context as a 5-word window around the target word

CPU times: user 25 s, sys: 216 ms, total: 25.2 s
Wall time: 8.78 s


In [10]:
%%time
# now you can get word vectors !
vec = model.wv.get_vector('anything')
print(type(vec))
print(vec.shape)
print(np.linalg.norm(vec))
print(vec)

<class 'numpy.ndarray'>
(32,)
11.120962
[-2.6667542  -0.48303333 -0.20257813  1.2679316   0.90690655  3.429711
  1.159343   -3.1437087   1.4900365   0.61652017 -0.7696336   0.47982252
  0.08071095 -0.6607585  -0.48443276  3.0956202  -2.4432104  -1.805039
  0.09163912 -6.0991135  -3.3911002  -0.6912671   1.1376634   1.3509796
  0.26996496 -0.09721832  0.3113912  -2.048509    1.348736    0.43299496
 -1.903464   -2.3767653 ]
CPU times: user 573 µs, sys: 6 µs, total: 579 µs
Wall time: 625 µs


In [11]:
model.wv.most_similar?

Signature:
model.wv.most_similar(
    positive=None,
    negative=None,
    topn=10,
    clip_start=0,
    clip_end=None,
    restrict_vocab=None,
    indexer=None,
)
Docstring:
Find the top-N most similar keys.
Positive keys contribute positively towards the similarity, negative keys negatively.

This method computes cosine similarity between a simple mean of the projection
weight vectors of the given keys and the vectors for each key in the model.
The method corresponds to the `word-analogy` and `distance` scripts in the original
word2vec implementation.

Parameters
----------
positive : list of (str or int or ndarray) or list of ((str,float) or (int,float) or (ndarray,float)), optional
    List of keys that contribute positively. If tuple, second element specifies the weight (default `1.0`)
negative : list of (str or int or ndarray) or list of ((str,float) or (int,float) or (ndarray,float)), optional
    List of keys that contribute negatively. If tuple, second element specifies the

In [12]:
# or query similar words directly. Go play with it!
model.wv.most_similar('bread')

[('rice', 0.9591684341430664),
 ('beans', 0.9364587664604187),
 ('cheese', 0.9299008250236511),
 ('butter', 0.9273766279220581),
 ('potato', 0.9235799312591553),
 ('sauce', 0.9231544733047485),
 ('chocolate', 0.916504442691803),
 ('pasta', 0.9157123565673828),
 ('wine', 0.9127898812294006),
 ('banana', 0.9103403091430664)]

In [13]:
vec_rice = model.wv.get_vector("rice")
vec_bread = model.wv.get_vector("bread")
print(np.linalg.norm(vec_rice))
print(np.linalg.norm(vec_bread))
cos_sim = np.dot(vec_rice, vec_bread) / (np.linalg.norm(vec_rice) * np.linalg.norm(vec_bread))
print(cos_sim)

6.74712
5.15931
0.9591685


In [14]:
vec_flour = model.wv.get_vector("flour")
vec_diff = vec_bread - vec_flour
model.wv.similar_by_vector(vec_diff)

[('panda', 0.584997296333313),
 ('musk', 0.5788779854774475),
 ('marijuana', 0.5582491755485535),
 ('it', 0.5486029386520386),
 ('grass', 0.5389667749404907),
 ('sense', 0.537616491317749),
 ('mcdonald', 0.5315647721290588),
 ('bodybuilding', 0.5269609689712524),
 ('taste', 0.5186756253242493),
 ('mood', 0.5172812938690186)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [3]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [4]:
print(type(model))

<class 'gensim.models.keyedvectors.KeyedVectors'>


In [5]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920561909675598),
 ('part-time', 0.4912828207015991),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.47964534163475037)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [6]:
words = model.index_to_key[:1000] 

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [7]:
print(type(words))
print(len(words))

print(words[::100])

<class 'list'>
1000
['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [8]:
# for each word, compute it's vector with model
word_vectors_array = [model.get_vector(word) for word in words]
word_vectors = np.vstack(word_vectors_array)

In [9]:
print(word_vectors.shape)
print(word_vectors.ndim)

(1000, 100)
2


In [10]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [11]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
pca = PCA(n_components=2, whiten=True)

# and maybe MORE OF YOUR CODE here :)
pca.fit(word_vectors)

word_vectors_pca = pca.transform(word_vectors)

In [12]:
print(word_vectors_pca.mean(0))
print(word_vectors_pca.std(0))

[-1.1163354e-06 -3.7100912e-07]
[0.99949926 0.9995001 ]


In [25]:
print(pca.components_.shape)

(2, 100)


In [26]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [13]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [14]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1004', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [15]:
%%time
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca


word_tsne = TSNE(n_components=2).fit_transform(word_vectors)

CPU times: user 5.72 s, sys: 314 ms, total: 6.04 s
Wall time: 1.3 s


In [43]:
normalized_word_tsne=(word_tsne - word_tsne.mean(0)) / word_tsne.std(0)

In [44]:
print(normalized_word_tsne.mean(0))
print(normalized_word_tsne.std(0))

[-1.9073487e-09 -7.1525572e-09]
[0.9999998 1.0000004]


In [46]:
draw_vectors(normalized_word_tsne[:, 0], normalized_word_tsne[:, 1], color='green', token=words)

figure(id='p1151', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [16]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    # YOUR CODE
    phrase = phrase.lower()
    # tokens = word_tokenize(phrase)
    tokens = tokenizer.tokenize(phrase)
    # print(tokens)
    count = 0

    for token in tokens:
        if token in model.key_to_index:
            vector += model.get_vector(token)
            count += 1

    # print(f"count = {count}")
    if count != 0:
        vector /= count
    
    return vector

In [19]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

print(vector)

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

[ 0.31807372  0.25188074  0.06748973  0.07918166 -0.20551686  0.3099363
  0.12886    -0.06323332  0.1167265   0.1529691  -0.02558171 -0.22829592
 -4.4815      0.02976158 -0.12603833 -0.1181161  -0.23754632 -0.02389293
 -0.47764084 -0.03727125  0.0933293  -0.11675651 -0.07968461  0.16103767
  0.22593975 -0.9288166  -0.13147809 -0.27092585  0.32907042 -0.12769467
 -0.1002182   0.05345418 -0.33383527  0.16147001  0.05949384  0.13145241
  0.04188244 -0.11879817  0.09030958 -0.02781658 -1.0278689   0.00457059
  0.21854158 -0.06651367  0.23628668 -0.14505692  0.17416383 -0.08423841
 -0.12737857 -0.01618933 -0.16621883  0.12946318  0.17684542 -0.01754692
  0.12413007  0.01527582 -0.11908681  0.11422     0.15738223  0.09719244
  0.05083408  0.0163896   0.03656067 -0.11753249  0.25182876 -0.10408742
 -0.11623914 -0.11769509 -0.22145198  0.07970998  0.17989802  0.05959851
  0.21143325 -0.06852868  0.09059133 -0.03333633 -0.12827642 -0.02455183
 -0.21025817 -0.0852936   1.3701859   0.28880623 -0.

In [83]:
%%time
# let's only consider ~5k phrases for a first run.
chosen_phrases = data  # data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors_array = [get_phrase_embedding(phrase) for phrase in chosen_phrases]
phrase_vectors = np.vstack(phrase_vectors_array)

CPU times: user 6.19 s, sys: 133 ms, total: 6.32 s
Wall time: 6.33 s


In [84]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [ ]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [23]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=7,)

figure(id='p1053', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [24]:
%%time
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

CPU times: user 6.16 s, sys: 74.4 ms, total: 6.24 s
Wall time: 6.25 s


In [27]:
print(data_vectors.shape)

(537272, 100)


In [34]:
def cos_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [40]:
embedding = get_phrase_embedding("How do i enter the matrix?")
print(cos_similarity(embedding, data_vectors[4]))
print(data[4])

0.90039605
Why was George RR Martin critical of JK Rowling after losing the Hugo award?



In [43]:
distances = np.apply_along_axis(lambda x: cos_similarity(embedding, x), axis=1, arr=data_vectors)
distances.shape

/tmp/ipykernel_6865/1204915391.py:2: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


(537272,)

In [48]:
distances

array([0.01626949, 0.92775255, 0.88651234, ..., 0.94756234, 0.9284008 ,
       0.9100322 ], dtype=float32)

In [62]:
nearest_indices = np.argpartition(-distances, kth=9)[:10]
print(nearest_indices)
# print(data[nearest_indices, :])
535949 in nearest_indices

[535949 248982 348630 194211 416639 358896 384831 277540 499334  78818]


True

In [71]:
nearest_distances = distances[nearest_indices]
print(nearest_distances)
sorted_level2_indices = np.argsort(-nearest_distances)
print(sorted_level2_indices)

[0.97950155 0.97944367 0.98048705 0.9796028  0.97943026 0.97942114
 0.9792139  0.9786363  0.978857   0.9781531 ]
[2 3 0 1 4 5 6 8 7 9]


In [73]:
[data[idx] for idx in nearest_indices[sorted_level2_indices]]

['How do I get to the dark web?\n',
 'What should I do to enter hollywood?\n',
 'How do I use the Greenify app?\n',
 'What can I do to save the world?\n',
 'How do I win this?\n',
 'How do I think out of the box? How do I learn to think out of the box?\n',
 'How do I find the 5th dimension?\n',
 'How do I use the pad in MMA?\n',
 'How do I estimate the competition?\n',
 'What do I do to enter the line of event management?\n']

In [53]:
nearest = distances[np.argpartition(-distances, kth=9)][:10]
# nearest_sorted = 

In [56]:
nearest[np.argsort(-nearest)]

array([0.98048705, 0.9796028 , 0.97950155, 0.97944367, 0.97943026,
       0.97942114, 0.9792139 , 0.978857  , 0.9786363 , 0.9781531 ],
      dtype=float32)

In [74]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    # YOUR CODE
    embedding = get_phrase_embedding(query)
    distances = np.apply_along_axis(lambda x: cos_similarity(embedding, x), axis=1, arr=data_vectors)
    nearest_indices = np.argpartition(-distances, kth=k - 1)[:k]
    nearest_distances = distances[nearest_indices]
    sorted_level2_indices = np.argsort(-nearest_distances)
    topk_lines = [data[idx] for idx in nearest_indices[sorted_level2_indices]]
    
    
    return topk_lines

In [75]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

/tmp/ipykernel_6865/1204915391.py:2: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



In [77]:
find_nearest(query="How does Trump?", k=10)

/tmp/ipykernel_6865/1204915391.py:2: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


['What does Donald Trump think about Israel?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of India?\n',
 'What does India think of Donald Trump?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [ ]:
find_nearest(query="Why don't i ask a question myself?", k=10)

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize `find_nearest` with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.